# DrainageAI Demo - Google Colab Integration with BYOL

This notebook demonstrates the DrainageAI workflow using Google Colab's GPU acceleration, including the new BYOL approach for few-shot learning and SAR data integration.

## Step 1: Check GPU Availability

In [ ]:
import torch

print(f"GPU available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. Processing will be slow.")

## Step 2: Install Dependencies

In [ ]:
!pip install rasterio geopandas scikit-image matplotlib pytorch-lightning torch-geometric

## Step 3: Clone the DrainageAI Repository

In [ ]:
# Clone the repository
!git clone https://github.com/yourusername/DrainageAI.git

%cd DrainageAI

## Step 4: Upload Test Imagery

In [ ]:
from google.colab import files

print("Please upload your multispectral imagery file (GeoTIFF format):")
uploaded = files.upload()

# Get the filename of the uploaded file
imagery_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {imagery_filename}")

## Step 5: Create Output Directory

In [ ]:
!mkdir -p colab_results

## Step 6: Calculate Spectral Indices

In [ ]:
print("\n=== Step 1: Calculate Spectral Indices ===\n")

!python main.py indices --imagery {imagery_filename} --output colab_results/indices.tif --indices ndvi,ndmi,msavi2

## Step 7: Run Drainage Detection

In [ ]:
print("\n=== Step 2: Detect Drainage Pipes ===\n")

# Choose one of the following model options:

# Semi-supervised model (default)
!python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_semi.tif --model semi

# Uncomment to use BYOL model (new)
# !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_byol.tif --model byol

## Step 8: Vectorize Results

In [ ]:
print("\n=== Step 3: Vectorize Results ===\n")

# Change the input file if you used a different model
detection_file = "colab_results/drainage_semi.tif"

!python main.py vectorize --input {detection_file} --output colab_results/drainage_lines.shp

## NEW: BYOL Workflow for Few-Shot Learning

This section demonstrates the BYOL (Bootstrap Your Own Latent) approach for scenarios with very few labeled examples.

### Upload Unlabeled and Labeled Data

In [ ]:
# Create directories for unlabeled and labeled data
!mkdir -p data/unlabeled/imagery
!mkdir -p data/labeled/imagery
!mkdir -p data/labeled/labels

# Upload unlabeled imagery (you can upload multiple files)
print("Please upload unlabeled imagery files (GeoTIFF format):")
uploaded_unlabeled = files.upload()

# Save uploaded files to the unlabeled directory
for filename in uploaded_unlabeled.keys():
    with open(f"data/unlabeled/imagery/{filename}", 'wb') as f:
        f.write(uploaded_unlabeled[filename])
    print(f"Saved {filename} to data/unlabeled/imagery/")

# Upload labeled imagery (you can upload multiple files)
print("\nPlease upload labeled imagery files (GeoTIFF format):")
uploaded_labeled_imagery = files.upload()

# Save uploaded files to the labeled imagery directory
for filename in uploaded_labeled_imagery.keys():
    with open(f"data/labeled/imagery/{filename}", 'wb') as f:
        f.write(uploaded_labeled_imagery[filename])
    print(f"Saved {filename} to data/labeled/imagery/")

# Upload label masks (you can upload multiple files)
print("\nPlease upload label mask files (GeoTIFF format):")
uploaded_labels = files.upload()

# Save uploaded files to the labeled labels directory
for filename in uploaded_labels.keys():
    with open(f"data/labeled/labels/{filename}", 'wb') as f:
        f.write(uploaded_labels[filename])
    print(f"Saved {filename} to data/labeled/labels/")

### BYOL Pretraining

In [ ]:
print("\n=== BYOL Pretraining ===\n")

# Run BYOL pretraining
!python examples/byol_mvp_workflow.py \
    --optical-dir data/unlabeled/imagery \
    --output-dir colab_results \
    --byol-epochs 20 \
    --inference-only False

### BYOL Fine-tuning

In [ ]:
print("\n=== BYOL Fine-tuning ===\n")

# Run BYOL fine-tuning
!python examples/byol_mvp_workflow.py \
    --optical-dir data/labeled/imagery \
    --label-dir data/labeled/labels \
    --output-dir colab_results \
    --model-path colab_results/byol_pretrained.pth \
    --num-labeled 5 \
    --finetune-epochs 10

### BYOL Inference

In [ ]:
print("\n=== BYOL Inference ===\n")

# Run inference with the fine-tuned BYOL model
!python examples/byol_mvp_workflow.py \
    --inference-only \
    --model-path colab_results/byol_finetuned.pth \
    --test-image {imagery_filename} \
    --output-dir colab_results

## NEW: SAR Data Integration

### Upload SAR Imagery

In [ ]:
print("Please upload your SAR imagery file (GeoTIFF format):")
uploaded_sar = files.upload()

# Get the filename of the uploaded file
sar_filename = list(uploaded_sar.keys())[0]
print(f"Uploaded SAR file: {sar_filename}")

### Run SAR Integration Example

In [ ]:
print("\n=== SAR Integration ===\n")

# Run SAR integration example
!python examples/sar_integration_example.py \
    --imagery {imagery_filename} \
    --sar {sar_filename} \
    --output-dir colab_results \
    --visualize

## Combining BYOL and SAR for Maximum Performance

In [ ]:
print("\n=== BYOL + SAR Integration ===\n")

# Run BYOL with SAR data
!python examples/byol_mvp_workflow.py \
    --inference-only \
    --model-path colab_results/byol_finetuned.pth \
    --test-image {imagery_filename} \
    --test-sar {sar_filename} \
    --output-dir colab_results